# C-Support Vector Classification

In [1]:
# libraries
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import imageio.v3 as imageio
import cv2

In [2]:
# Get directories
train_dir = "C:/Users/bodin/School/MATH 5680/Projec/tumor_images/Training"
test_dir = "C:/Users/bodin/School/MATH 5680/Projec/tumor_images/Testing"

labels = ["glioma_tumor","meningioma_tumor","no_tumor","pituitary_tumor"]

In [3]:
# Create list of data
train_img_list = []
test_img_list = []

test_label_list = []
train_label_list = []

# Training dataset
for label in labels:
    for img_file in os.listdir(train_dir + '/' + label):
        img = cv2.imread(train_dir + '/' + label + '/' + img_file)
        img = cv2.resize(img, (200,200))
        train_img_list.append(img)
        train_label_list.append(label)

# Testing dataset
for label in labels:
    for img_file in os.listdir(test_dir + '/' + label):
        img = cv2.imread(test_dir + '/' + label + '/' + img_file)
        img = cv2.resize(img, (200,200))
        test_img_list.append(img)
        test_label_list.append(label)

In [4]:
# Encode labels
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

train_label_list = label_encoder.fit_transform(train_label_list)
test_label_list = label_encoder.fit_transform(test_label_list)

# Convert images to int64 array
train_img_list = np.array(train_img_list).astype(np.int64)
test_img_list = np.array(test_img_list).astype(np.int64)

In [5]:
# Value Counts for each type of tumor
"""
Pituitary = 3
Glioma = 0
Meningioma = 1
None = 2
"""
pd.Series(train_label_list).value_counts()

3    827
0    826
1    822
2    395
dtype: int64

In [6]:
# Convert images from (200, 200, 3) to column vector (120000)
X_train = train_img_list.reshape(len(train_img_list), -1)
X_test = test_img_list.reshape(len(test_img_list), -1)
Y_train = train_label_list
Y_test = test_label_list

In [7]:
# Standardize values between 0-1
X_train = X_train / 255
X_test = X_test / 255

In [8]:
### Not necessary in current state, may delete ###

#from sklearn.decomposition import PCA
#pca = PCA(.5)

#pca_train = pca.fit_transform(X_train)
#pca_test = pca.transform(X_test)

#pca_train = X_train
#pca_test = X_test

In [14]:
from sklearn.svm import SVC

"""
SVC parameters to alter:

C: Regularization parameter. The strength of the regularization 
is inversely proportional to C. Must be strictly positive. 
The penalty is a squared l2 penalty. [default=1.0]

kernel: Specifies the kernel type to be used in the algorithm.
{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}
[default=rbf]

degree: Degree of the polynomial kernel function (‘poly’)
[default = 3]

gamma: Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’.
[default = 'scale']

tol: Tolerance for stopping criterion.
[default = 1e-3]

"""

sv = SVC(max_iter=100)
sv.fit(X_train, Y_train)

C:\Users\bodin\anaconda3\lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


SVC(max_iter=100)

In [15]:
# Evaluation
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)

y_pred = sv.predict(X_test)
accuracy = accuracy_score(Y_test, y_pred)

print("Training Score: ", sv.score(X_train, Y_train))
print("testing Score: ", sv.score(X_test, Y_test))
print(f"Current hyperparameter: {sv.get_params()['C']}")
print(f'Accuracy: {accuracy}\n')
print(f"Confusion Matrix: \n", confusion_matrix(Y_test, y_pred), "\n")
print(classification_report(Y_test, y_pred))

Training Score:  0.8174216027874565
testing Score:  0.5126903553299492
Current hyperparameter: 1.0
Accuracy: 0.5126903553299492

Confusion Matrix: 
 [[25 23 46  6]
 [34 53 28  0]
 [15  8 82  0]
 [ 1 16 15 42]] 

              precision    recall  f1-score   support

           0       0.33      0.25      0.29       100
           1       0.53      0.46      0.49       115
           2       0.48      0.78      0.59       105
           3       0.88      0.57      0.69        74

    accuracy                           0.51       394
   macro avg       0.55      0.51      0.52       394
weighted avg       0.53      0.51      0.50       394

